In [11]:
!pip install requests
!pip install rcsbsearchapi

In [12]:
import re
import requests
import json

from rcsbsearchapi.search import SequenceQuery

### Phase 1: Solicit User for UniProt ID and Query/Save the Protein's Similar Sequences

In [13]:
import requests

def is_uniprot(pid):
    """
    Check if a UniProt ID exists in the UniProt database.

    Parameters:
    pid (str): The UniProt ID to verify.

    Returns:
    bool: True if the UniProt ID exists, False otherwise.
    """
    uniprot_api = f"https://www.uniprot.org/uniprot/{pid}.txt"
    response = requests.get(uniprot_api)
    return response.status_code == 200

def get_fasta(pid):
    """
    Retrieve the FASTA formatted sequence for a given UniProt ID.

    Parameters:
    pid (str): The UniProt ID for which to fetch the FASTA sequence.

    Returns:
    str: The FASTA formatted sequence if the UniProt ID exists, otherwise an error message.
    """
    uniprot_fasta_url = f"https://www.uniprot.org/uniprot/{pid}.fasta"
    response = requests.get(uniprot_fasta_url)
    if response.status_code == 200:
        return response.text
    else:
        return f"Error: Unable to retrieve FASTA for UniProt ID {pid}"


In [14]:
def fetch_similarpid(fasta_sequence, pid):
    """
    Performs a sequence similarity search against the RCSB Protein Data Bank (PDB) using a FASTA sequence.

    Parameters:
    fasta_sequence (str): The FASTA formatted sequence of a protein to search for.
    evalue_cutoff (float): The e-value cutoff for the search. Default is 1.
    identity_cutoff (float): The minimum identity percentage for the search. Default is 0.9.

    Returns:
    arr: The search result as an array of PDB IDs.
    """

    search_request = {
        "query": {
            "type": "terminal",
            "service": "sequence",
            "parameters": {
            "evalue_cutoff": 0.0001,
            "identity_cutoff": 0.6,
            "sequence_type": "protein",
            "value": fasta_sequence
            }
        },
        "return_type": "polymer_entity",
        "request_options": {
            "paginate": {
            "start": 0,
            "rows": 1000
            },
            "results_content_type": [
            "experimental"
            ],
            "sort": [
            {
                "sort_by": "score",
                "direction": "desc"
            }
            ],
            "scoring_strategy": "combined"
        }
    }   

    # The json parameter in the requests.post automatically converts the Python dictionary to a JSON payload.
    response = requests.post("https://search.rcsb.org/rcsbsearch/v2/query", json=search_request)

    # JSON elements are turned into a list    
    identifiers_json = response.json()
    identifiers = [result["identifier"] for result in identifiers_json["result_set"]]

    print(identifiers)
    return identifiers


In [15]:
def fetch_inchi_keys(pdb_id):
    """
    Fetch InChIKeys for non-polymer entities (small molecules) associated with a PDB ID.

    Parameters:
    pdb_id (str): A string representing the PDB ID for which to fetch the InChIKeys.

    Returns:
    dict: A dictionary object that contains the response with InChIKeys.
    """
    match = re.match(r'(\w+)_([0-9]+)', pdb_id)

    if match:
        pdb_id = match.group(1)
        num = match.group(2)
    else:
        raise Exception(f"Invalid PDB ID: {pdb_id}")
    
    url = 'https://data.rcsb.org/graphql'
    headers = {'Content-Type': 'application/json'}
    query = """
    query Structure($id: String!) {
      entry(entry_id: $id) {
        rcsb_id
        polymer_entities {
          rcsb_polymer_entity_container_identifiers {
            uniprot_ids
            reference_sequence_identifiers {
              database_accession
            }
          }
        }
        nonpolymer_entities {
          nonpolymer_comp {
            rcsb_chem_comp_descriptor {
              InChIKey
            }
          }
        }
      }
    }
    """
    variables = {'id': pdb_id}
    response = requests.post(url, headers=headers, json={'query': query, 'variables': variables})
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Query failed to run by returning code of {response.status_code}. {response.text}")

In [16]:
def process_pdb_results(pdb_results):
    """
    Process a list of PDB IDs by fetching related data for each ID and compiling it into a list of tuples.

    Each tuple contains the RCSB ID, a set of UniProt IDs, and a set of InChIKeys associated with the PDB ID.
    This function assumes that each PDB ID corresponds to one entry in the RCSB database and that
    the `fetch_inchi_keys` function is defined elsewhere to return the appropriate JSON structure.

    Parameters:
    pdb_results (list of str): A list of PDB IDs to process.

    Returns:
    list of tuples: A list where each tuple contains (RCSB ID, UniProt ID, InChIKey).
    """
    
    results_tuples = []

    for pdb_id in pdb_results:
        json_data = fetch_inchi_keys(pdb_id)['data']['entry']

        rcsb_id = json_data['rcsb_id']

        # Extract UniProt IDs
        uniprot_ids = set()
        for entity in json_data.get('polymer_entities', []):
            container_identifiers = entity.get('rcsb_polymer_entity_container_identifiers', {})
            identifiers = container_identifiers.get('reference_sequence_identifiers')
            if identifiers:  # This check ensures that identifiers is not None
                for identifier in identifiers:
                    uniprot_ids.add(identifier.get('database_accession', ''))  # Use .get() for safe access


        # Extract InChIKeys
        inchi_keys = set()
        for entity in json_data.get('nonpolymer_entities', []):
            if 'rcsb_chem_comp_descriptor' in entity.get('nonpolymer_comp', {}):
                inchi_keys.add(entity['nonpolymer_comp']['rcsb_chem_comp_descriptor']['InChIKey'])

        # Combine the extracted data into tuples (assuming one InChIKey for simplicity)
        for uniprot_id in uniprot_ids:
            inchi_key = next(iter(inchi_keys)) if inchi_keys else ''
            results_tuples.append((rcsb_id, uniprot_id, inchi_key))

    return results_tuples


### Phase 2: Query All Assays Related to all Identified Proteins from PubChem BioAssay

In [17]:
def find_matching_aids(json_uniprot, json_cid):
    
    # Extract the AID lists from the provided JSON objects
    aid_list_uniprot = json_uniprot['IdentifierList']['AID']
    
    # Since there can be multiple entries in the 'Information' list, we will iterate through each to collect all AID's
    aid_list_cid = []
    for info in json_cid['InformationList']['Information']:
        aid_list_cid.extend(info['AID'])
    
    # Finding the intersection of both AID lists to get the array of matching AIDs
    matching_aids = list(set(aid_list_uniprot) & set(aid_list_cid))
    
    return matching_aids


In [18]:
def pubchem_match(processed_results):

    pubchem_match = [-1,-1]
    valid_results = []

    for result in processed_results:
        uniprot_id = result[1]
        inchi_key = result[2]

        new_tuple = (uniprot_id, inchi_key, -1, -1, -1) # New tuple with (UniProt ID, InChI key, CID, AID)
        temp_array = [-1,-1, -1] # [CID, AIDs for CID, AIDs for UniProt ID]

        pubchem_api = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{inchi_key}/cids/JSON" # This section works
        response = requests.get(pubchem_api)
        if response.status_code == 200:
            data = response.json()
            cid = data['IdentifierList']['CID'][0]
            temp_array[0] = cid
        else:
            print(f"No valid CID for InChI key: {inchi_key}")

        pubchem_api = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/aids/JSON" # This section doesn't work
        response = requests.get(pubchem_api)
        if response.status_code == 200:
            aid_cid = response.json() # Need to add an element here that will return an array of AIDs
            temp_array[1] = aid_cid
        else:
            print(f"No valid AID for Compound: {cid}")

        pubchem_api = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/target/proteinname/{uniprot_id}/aids/JSON" 
        response = requests.get(pubchem_api)
        if response.status_code == 200:
            aid_uniprot = response.json() # Need to add an element here that will return an array of AIDs
            temp_array[2] = aid_uniprot 
        else:
            print(f"Error: Unable to retrieve AID for UniProt ID {uniprot_id}")

        # before adding, we need to check that there is a match between the aid_uniprot and aid_cid
        matching_aid = find_matching_aids(aid_uniprot, aid_cid)
        
        if temp_array[0] != -1 and temp_array[1] != -1 and temp_array[2] != -1 and len(matching_aid) > 0:
            new_tuple = (uniprot_id, inchi_key, temp_array[0], matching_aid)
            valid_results.append(new_tuple)

    return valid_results


### Phase 5: Output Remaining Data as a Table

In [19]:
#### PROGRAM EXECUTION ####

def main():
    pid = input("Enter a Uniprot ID: ")
    if is_uniprot(pid):
        fasta = get_fasta(pid)
        if fasta:
            # Extract the sequence from the FASTA format
            sequence = re.search(r'(?<=\n)[A-Z\n]+', fasta)
            fasta_sequence = sequence.group(0).replace("\n", "")
            
            # Fetch similar protein IDs based on the FASTA sequence. Array is returned.
            pdb_results = fetch_similarpid(fasta_sequence, pid)
            print("Sequence 1 Complete")

        else:
            raise Exception("No FASTA sequence was retrieved.")
        
        # Extract inChI keys from PDB IDs from the fetch_similarpid results
        processed_results = process_pdb_results(pdb_results)
        print("Sequence 2 Complete")
        print(processed_results)

        print(pubchem_match(processed_results))
        print("Sequence 3 Complete")
    
    else:
        print(f"Error: {pid} does not exist in UniProt.")

if __name__ == "__main__":
    main()

['1YY4_1', '1YYE_1', '1QKM_1', '1L2J_1', '2I0G_1', '2JJ3_1', '2QTU_1', '2Z4B_1', '4ZI1_1', '5TOA_1', '1NDE_1', '1U3R_1', '1U9E_1', '2GIU_1', '1U3Q_1', '1U3S_1', '1X76_1', '1X78_1', '1X7B_1', '1X7J_1', '2YJD_1', '3OLL_1', '3OLS_1', '3OMO_1', '3OMP_1', '3OMQ_1', '4J24_1', '4J26_1', '2FSZ_1', '2NV7_1', '2YLY_1', '1ZAF_1', '2J7X_1', '2J7Y_1', '1HJ1_1', '1QKN_1', '7XVY_1', '7XVZ_1', '7XWP_1', '7XWQ_1', '7XWR_1', '7NDO_1', '7NEL_1', '7NFB_1', '1HCQ_3', '1HCP_1', '4AA6_1', '4OLN_1', '8IFO_1', '1LO1_3', '8CEF_3', '4OND_1', '1CIT_3', '6LC1_2']
Sequence 1 Complete
Sequence 2 Complete
[('1YY4', 'Q92731', 'YHEHVRSGKUYDON-UHFFFAOYSA-N'), ('1YY4', 'Q15788', 'YHEHVRSGKUYDON-UHFFFAOYSA-N'), ('1YYE', 'Q92731', 'NSSOSHDCWCMNDM-UHFFFAOYSA-N'), ('1QKM', 'Q92731', 'TZBJGXHYKVUXJN-UHFFFAOYSA-N'), ('1L2J', 'Q92731', 'MASYAWHPJCQLSW-ZIAGYGMSSA-N'), ('2I0G', 'Q92731', 'XIESSJVMWNJCGZ-VKJFTORMSA-N'), ('2JJ3', 'Q92731', 'RHQLNMNKTIOREN-AOIWGVFYSA-N'), ('2QTU', 'Q92731', 'GPFRMIHXGMVMGF-BZSNNMDCSA-N'), ('2Z4B', '